In [63]:
from scapy.all import rdpcap
from scapy.layers.dot11 import Dot11Elt

In [64]:
capture = rdpcap("../../data/raw/TEST/OppoFindX3Neo_A.pcap")

In [65]:
EXTENDED_CAP = {
    0: "20_40_BSS_Coexistence_Management_Support",
    1: "Reserved_1",
    2: "Extended_Channel_Switching",
    3: "Reserved_2",
    4: "PSMP_Capability",
    5: "Reserved_3",
    6: "SPSMP_Support",
    7: "Event",
    8: "Diagnostics",
    9: "Multicast_Diagnostic",
    10: "Location_Tracking",
    11: "FMS",
    12: "Proxy_Arp_Service",
    13: "Collocated_Interference_Reporting",
    14: "Civic_Location",
    15: "Geospatial_Location",
    16: "TFS",
    17: "WNM_Sleep_Mode",
    18: "TIM_Broadcast",
    19: "BSS_Transition",
    20: "QoS_Traffic_Capability",
    21: "AC_Station_Count",
    22: "Multiple_BSSID",
    23: "Timing_Measurement",
    24: "Channel_Usage",
    25: "SSID_List",
    26: "DMS",
    27: "UTC_Timing",
    28: "TPU_Buffer_STA_Support",
    29: "TDLS_Peer_PSM",
    30: "TDLS_Channel_Switching",
    31: "Interworking",
    32: "QoS_Map",
    33: "EBR",
    34: "SSPN_Interface",
    35: "Reserved_4",
    36: "MSGCF_Capability",
    37: "TDLS_Support",
    38: "TDLS_Prohibited",
    39: "TDLS_Channel_Switching_Prohibited",
    40: "Reject_Unadmitted_Frame",
    41: "Service_Interval_Granularity",
    42: "Service_Interval_Granularity",
    43: "Service_Interval_Granularity",
    44: "Identifier_Location",
    45: "U_APSD_Coexistence",
    46: "WNM_Notification",
    47: "QAB_Capability",
    48: "UTF_8_SSID",
    49: "QMF_Activated",
    50: "QMF_ReconfigurationActivated",
    51: "Robust_AV_Streaming",
    52: "Advanced_GCR",
    53: "Mesh_GCR",
    54: "SCS",
    55: "QLoad_Report",
    56: "Alternate_EDCA",
    57: "Unprotected_TXOP",
    58: "Protected_RXOP",
    59: "Reserved_5",
    60: "Protected_QLoad_Report",
    61: "TDLS_Wide_Bandwidth",
    62: "Operating_Mode_Notification",
    63: "Max_Number_Of_MSDUs_In_A_MPDU",
    64: "Max_Number_Of_MSDUs_In_A_MPDU",
    65: "Channel_Schedule_Management",
    66: "Geodatabase_Inband_Enabling_Signal",
    67: "Network_Channel_Control",
    68: "White_Space_Map",
    69: "Channel_Availability_Query",
    70: "FTM_Responder",
    71: "FTM_Initiator",
    72: "Reserved_6",
    73: "ESM_Capability",
    74: "Future_Channel_Guidance",
}

In [66]:
capture[0]

<RadioTap  version=0 pad=0 len=15 present=Flags+Rate+Channel+dBm_AntSignal Flags=FCS Rate=5.5 Mbps ChannelFrequency=2412 ChannelFlags=CCK+2GHz dBm_AntSignal=-50 dBm notdecoded='' |<Dot11FCS  subtype=Probe Request type=Management proto=0 FCfield= ID=0 addr1=ff:ff:ff:ff:ff:ff (RA=DA) addr2=22:e4:72:fb:91:70 (TA=SA) addr3=ff:ff:ff:ff:ff:ff (BSSID/STA) SC=32560 fcs=0x28ac8618 |<Dot11ProbeReq  |<Dot11Elt  ID=SSID len=0 info='' |<Dot11EltRates  ID=Supported Rates len=4 rates=[1.0 Mbps, 2.0 Mbps, 5.5 Mbps, 11.0 Mbps] |<Dot11EltRates  ID=Extended Supported Rates len=8 rates=[6.0 Mbps, 9.0 Mbps, 12.0 Mbps, 18.0 Mbps, 24.0 Mbps, 36.0 Mbps, 48.0 Mbps, 54.0 Mbps] |<Dot11EltDSSSet  ID=DSSS Set len=1 channel=1 |<Dot11EltHTCapabilities  ID=HT Capabilities len=26 L_SIG_TXOP_Protection=0 Forty_Mhz_Intolerant=0 PSMP=0 DSSS_CCK=0 Max_A_MSDU=3839 o Delayed_BlockAck=0 Rx_STBC=1 Tx_STBC=1 Short_GI_40Mhz=1 Short_GI_20Mhz=1 Green_Field=0 SM_Power_Save=disabled Supported_Channel_Width=20Mhz+40Mhz LDPC_Coding_C

In [67]:
packet = capture[0].getlayer(Dot11Elt, ID=127).info.hex()

In [68]:
packet

'00000a8200400000000120'

In [69]:
def hex_string_to_binary(hex_string):
    # Initialize the resulting binary string
    binary_string = ""

    # Iterate every two characters in the hexadecimal string
    for i in range(0, len(hex_string), 2):
        # Extract the pair of characters
        hex_pair = hex_string[i : i + 2]
        # Convert the pair to a byte (decimal) in little endian format
        byte = int(hex_pair, 16)
        # Reverse the byte
        byte = int("{:08b}".format(byte)[::-1], 2)
        # Convert the byte to a binary string of 8 bits and concatenate
        binary_string += f"{byte:08b}"

    return binary_string

In [70]:
bin_packet = hex_string_to_binary(packet)

In [71]:
bin_packet

'0000000000000000010100000100000100000000000000100000000000000000000000001000000000000100'

In [72]:
def extract_fields_from_binary(ie_dictionary, binary_string):
    # Initialize a list to hold tuples of field names and their corresponding bits
    extracted_fields = []

    # Convert the binary string to a list for easier access by index, and reverse it
    binary_list = list(binary_string)

    # Initialize a set to keep track of fields already added
    added_fields = set()

    # Iterate through each bit index in the extended capabilities dictionary
    for bit_index in range(len(binary_list)):
        # Check if this bit index is in the dictionary (to handle binary strings longer than the dictionary)
        if bit_index in ie_dictionary:
            field_name = ie_dictionary[bit_index]
            # If the field has not been added yet, proceed to extract its bits
            if field_name not in added_fields:
                # Find all bit indexes for this field
                bit_indexes = [
                    index for index, name in ie_dictionary.items() if name == field_name
                ]
                # Extract bits for the current field
                field_bits = "".join(
                    [
                        binary_list[index] if index < len(binary_list) else "0"
                        for index in bit_indexes
                    ]
                )
                # Add the field and its bits to the list
                extracted_fields.append((field_name, field_bits))
                # Mark this field as added
                added_fields.add(field_name)

    return extracted_fields

In [73]:
# Call the function and print the result
extracted_fields = extract_fields_from_binary(EXTENDED_CAP, bin_packet)
for field, bits in extracted_fields:
    print(f"{field}: {bits}")

20_40_BSS_Coexistence_Management_Support: 0
Reserved_1: 0
Extended_Channel_Switching: 0
Reserved_2: 0
PSMP_Capability: 0
Reserved_3: 0
SPSMP_Support: 0
Event: 0
Diagnostics: 0
Multicast_Diagnostic: 0
Location_Tracking: 0
FMS: 0
Proxy_Arp_Service: 0
Collocated_Interference_Reporting: 0
Civic_Location: 0
Geospatial_Location: 0
TFS: 0
WNM_Sleep_Mode: 1
TIM_Broadcast: 0
BSS_Transition: 1
QoS_Traffic_Capability: 0
AC_Station_Count: 0
Multiple_BSSID: 0
Timing_Measurement: 0
Channel_Usage: 0
SSID_List: 1
DMS: 0
UTC_Timing: 0
TPU_Buffer_STA_Support: 0
TDLS_Peer_PSM: 0
TDLS_Channel_Switching: 0
Interworking: 1
QoS_Map: 0
EBR: 0
SSPN_Interface: 0
Reserved_4: 0
MSGCF_Capability: 0
TDLS_Support: 0
TDLS_Prohibited: 0
TDLS_Channel_Switching_Prohibited: 0
Reject_Unadmitted_Frame: 0
Service_Interval_Granularity: 000
Identifier_Location: 0
U_APSD_Coexistence: 0
WNM_Notification: 1
QAB_Capability: 0
UTF_8_SSID: 0
QMF_Activated: 0
QMF_ReconfigurationActivated: 0
Robust_AV_Streaming: 0
Advanced_GCR: 0
Mes

In [74]:
vendor_packet = capture[0]

In [98]:
from scapy.all import *
from scapy.layers.dot11 import Dot11EltVendorSpecific


def extract_vendor_specific_elements(packet):
    # List to store all Dot11EltVendorSpecific elements
    vendor_specific_elements = []

    # Iterate through the layers of the packet
    for layer in packet.layers():
        print(layer)
        # Check if the layer is Dot11EltVendorSpecific
        if str(layer) == "<class 'scapy.layers.dot11.Dot11EltVendorSpecific'>":
            print("check")
            # Add the element to the list
            vendor_specific_elements.append(layer.oui)

    return vendor_specific_elements


# Example usage:
# Assuming 'packet' is your packet captured using Scapy
vendor_specific_elements = extract_vendor_specific_elements(vendor_packet)

<class 'scapy.layers.dot11.RadioTap'>
<class 'scapy.layers.dot11.Dot11FCS'>
<class 'scapy.layers.dot11.Dot11ProbeReq'>
<class 'scapy.layers.dot11.Dot11Elt'>
<class 'scapy.layers.dot11.Dot11EltRates'>
<class 'scapy.layers.dot11.Dot11EltRates'>
<class 'scapy.layers.dot11.Dot11EltDSSSet'>
<class 'scapy.layers.dot11.Dot11EltHTCapabilities'>
<class 'scapy.layers.dot11.Dot11Elt'>
<class 'scapy.layers.dot11.Dot11EltVendorSpecific'>
check


AttributeError: 'OUIField' object has no attribute 'hex'

In [97]:
print(vendor_specific_elements)

[<OUIField (Dot11EltVendorSpecific,Dot11EltMicrosoftWPA).oui>, <OUIField (Dot11EltVendorSpecific,Dot11EltMicrosoftWPA).oui>, <OUIField (Dot11EltVendorSpecific,Dot11EltMicrosoftWPA).oui>]


In [ ]:
from scapy.all import *
from scapy.layers.dot11 import Dot11EltVendorSpecific


def extract_vendor_specific_elements(packet):
    # List to store all Dot11EltVendorSpecific elements
    vendor_specific_elements = []

    # Iterate through the layers of the packet
    for layer in packet.layers():
        # Check if the layer is Dot11EltVendorSpecific
        if isinstance(layer, Dot11EltVendorSpecific):
            # Add the element to the list
            vendor_specific_elements.append(layer)

    return vendor_specific_elements